The file format is as follows.  In each instance, the number of variables and the number of clauses is the same, and this number is specified on the first line of the file.  Each subsequent line specifies a clause via its two literals, with a number denoting the variable and a "-" sign denoting logical "not".  For example, the second line of the first data file is "-16808 75250", which indicates the clause \neg x_{16808} \vee x_{75250}¬x 
16808
​
 ∨x 
75250
​
 .

Your task is to determine which of the 6 instances are satisfiable, and which are unsatisfiable.  In the box below, enter a 6-bit string, where the ith bit should be 1 if the ith instance is satisfiable, and 0 otherwise.  For example, if you think that the first 3 instances are satisfiable and the last 3 are not, then you should enter the string 111000 in the box below.

DISCUSSION: This assignment is deliberately open-ended, and you can implement whichever 2SAT algorithm you want.  For example, 2SAT reduces to computing the strongly connected components of a suitable graph (with two vertices per variable and two directed edges per clause, you should think through the details).  This might be an especially attractive option for those of you who coded up an SCC algorithm in Part 2 of this specialization.  Alternatively, you can use Papadimitriou's randomized local search algorithm.  (The algorithm from lecture is probably too slow as stated, so you might want to make one or more simple modifications to it --- even if this means breaking the analysis given in lecture --- to ensure that it runs in a reasonable amount of time.)  A third approach is via backtracking.  In lecture we mentioned this approach only in passing; see Chapter 9 of the Dasgupta-Papadimitriou-Vazirani book, for example, for more details.

In [ ]:
import numpy as np
import time

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import sys
from collections import Counter
import time

# depth first search for ssc
def initializeDigraph(data):

#     data = np.loadtxt(filename,dtype=int)
    G = {}

    allUniqueNodes = np.unique(data)
    for row in range(len(data)):
        node = data[row,0]
        edge = data[row,1]
        if node in G.keys():
            nodeData = G[node]
            edgeList = nodeData[1]
            edgeList.append(edge)
            G.update({node:(False,edgeList)})
        else:
            G.update({node:(False,[edge])})
            
    for node in allUniqueNodes:
        if node not in G.keys():
            G.update({node:(False,[node])})
        
    return G


def reverseGraph(G):
    data = []
    for node,nodeData in G.items():
        for edge in nodeData[1]:
            data.append((edge,node))
    data = np.asarray(data,dtype=int)
    Grev = initializeDigraph(data)
    return Grev


def dfs_scc_stack_optimized(G,start,s,orderCounter,order,leader):
    stack = np.zeros((len(G)*10,),dtype=int)
    stack[0] = start
    stackCounter = 0
    while stackCounter >=0:
        node = stack[stackCounter]
        stackCounter -= 1
        if not G[node][0]:
            G.update({node:(True,G[node][1])})
            leader[node-1] = s
            stackCounter += 1
            stack[stackCounter] = node
            for edge in G[node][1][::-1]:
                if not G[edge][0]:
                    stackCounter += 1
                    stack[stackCounter] = edge
        else:
            if node not in order:
                order[orderCounter] = node
                orderCounter +=1
    return G,s,orderCounter,order,leader


# subroutine

def dfs_loop_stack_optimized(G,ordering=None):
    orderCounter = 0
    order = np.zeros((len(G,)),dtype=int)
    leader = np.zeros((len(G,)),dtype=int)
                     

    # For pass 1 use nodes of graph in reverse order, for pass 2 use ordering
    if ordering is None:
        nodes = sorted(G.keys(),reverse=True)
    else:
        nodes = ordering

    # Loop through nodes
    for node in nodes:
        if not G[node][0]:
            s = node
            leader[node-1] = s
            G,s,orderCounter,order,leader = dfs_scc_stack_optimized(G,node,s,orderCounter,order,leader)
    order = np.flip(order)
    return order,leader


def kosarajui_SSC_stack_initializePath(G):

    

    order,_ = dfs_loop_stack_optimized(reverseGraph(G.copy()))
    print(order)
    _,leader = dfs_loop_stack_optimized(G,ordering=order)
    
    
    return leader

In [ ]:
# Load data
fileName = '2sat1.txt'
fileName = '2sat2.txt'
fileName = '2sat3.txt'
# fileName = '2sat4.txt'
# fileName = '2sat5.txt'
# fileName = '2sat6.txt'
data = np.loadtxt(fileName,skiprows=1,dtype=int) #float
# data[:,0] = data[:,0]-1
n = len(data)  
print(n)
data1 = data.copy()
data1[:,0] = data[:,1]
data1[:,1]  = data[:,0]
data = np.concatenate((data,data1))
len(data)

In [11]:
# Read data
# nodes = np.unique(data)
# # Build 
# nodes = np.unique(np.concatenate((nodes,-1*nodes)))
G = initializeDigraph(data)

In [ ]:
numOfComponentsPerSCC = kosarajui_SSC_stack_initializePath(G)

In [ ]:
numOfComponentsPerSCC

In [ ]:
numOfComponentsPerSCC